# MILP Formulation from Nathan Kallus' Paper (Problem 4)

In [1]:
import gurobipy as gp
from gurobipy import GRB
import sys
import math
import numpy as np
import pandas as pd
import random

## Model specifications
#### Since we chose to modify the formulation to a certain extent, these variables simply allow us to revert back to the original model
- delta_include: If true, then constraint 4c from original formulation holds. If false, only the added constraint that gamma[p] need to add to 1 holds
- different_Cp = If true, then we have different sets of branching choices for every non-leaf node (like original formulation). If false, then we have a static set C for all nodes

In [2]:
# -- This class is an alternative to solving the right/left ancestor problem --
"""
INPUT: d = depth of tree (which includes root node), so d = 2 would make a tree with {1, 2, 3}
RELEVANT FUNCTIONS:
- get_right_left: For all leaf nodes, returns its right and left ancestors in a dictionary 
                  of {(p, q): 1 or -1 if q is right or left ancestor respectively}
"""
class Tree:
    def __init__(self, d):
        self.depth = d
        self.nodes = list(range(1, 2**(d-1)))
        self.leaves = list(range(2**(d-1), 2**d))
        self.ancestor_rl = {}
    
    def get_left_children(self, n):
        if n in self.nodes:
            return int(2*n)
        else:
            raise Exception ('Invalid node n')
    
    def get_right_children(self, n):
        if n in self.nodes:
            return int(2*n+1)
        else:
            raise Exception ('Invalid node n')
    
    def get_parent(self, n):
        if (n in self.nodes) | (n in self.leaves):
            return int(math.floor(n/2))
        else:
            raise Exception ('Invalide node n')
    
    def get_ancestors(self, direction, n):
        current = n
        ancestors = []
        while current != 1:
            current_buffer = self.get_parent(current)
            if direction == 'r':
                if self.get_right_children(current_buffer) == current:
                    ancestors.append(current_buffer)
            else:
                if self.get_left_children(current_buffer) == current:
                    ancestors.append(current_buffer)
            current = current_buffer
        return ancestors
    
    def get_right_left(self):
        for i in self.leaves:
            right = self.get_ancestors('r', i)
            for j in right:
                self.ancestor_rl[(i, j)] = 1
            left = self.get_ancestors('l', i)
            for j in left:
                self.ancestor_rl[(i, j)] = -1
        return self.ancestor_rl

# Evaluating Synthetic Data

In [3]:
def open_file(file_path):
    df = pd.read_csv(file_path)
    train_X = df.iloc[:, :20]
    real = df.iloc[:, 20:22]
    train_t = df.iloc[:, 22]
    train_y = df.iloc[:, 24]
    return train_X, train_t, train_y, real

In [5]:
def open_file_v2(file_path):
    df = pd.read_csv(file_path)
    train_X = df.iloc[:, :3]
    real = df.iloc[:, 3:5]
    train_t = df.iloc[:, 5]
    train_y = df.iloc[:, 7]
    return train_X, train_t, train_y, real

In [4]:
def datapoint_tree(node, i, test_X, test_real, test_t):
    if node in L: #if datapoint has reached leaf node, calculate error
        index = treatments[node]
        ideal_outcome = max(test_real.iloc[i, :])
        difference = ideal_outcome - test_real.iloc[i, index]
        if difference == 0:
            count_optimal = 1
        else:
            count_optimal = 0
        
        if index == test_t[i]:
            same_treatment = 1
        else:
            same_treatment = 0
        return difference, count_optimal, same_treatment
    if test_X.iloc[i, branching[node]] <= 0: # go left (node 2)
        return datapoint_tree(tree.get_left_children(node), i, test_X, test_real, test_t)
    else:
        return datapoint_tree(tree.get_right_children(node), i, test_X, test_real, test_t)

In [5]:
def get_metrics(test_X, test_real, test_t):
    difference = 0
    count_optimal = 0
    count_same = 0
    for i in range(len(test_X)):
        diff, optimal, treat = datapoint_tree(1, i, test_X, test_real, test_t)
        difference += diff
        count_optimal += optimal
        count_same += treat
    return difference, float(count_optimal)/len(test_X), float(count_same)/len(test_X)

## Declaring variables determined a-priori

### a) Specfiying Input to Model
- m treatments indexed by t {1,..., m}
- n datapoints indexed by i {(X1, T1, Y1), ..., (Xn, Tn, Yn)} 
- d: depth of decision tree
- n_min: minimum number of datapoints of each treatment in node p
- num_features
- num_cuts

In [6]:
def run_tree(depth, bert):
    delta_include = False
    different_Cp = False
    bertsimas = bert

    m = {0, 1}
    n = len(train_X)
    d = int(depth)
    n_min = 0
    num_features = 2
    num_cuts = 2

    """# ---- CONSTRUCTING COMPLETE BINARY TREE ----
    # - P = number of nodes in the tree
    # - L_c = set of non-leaf nodes
    # - L = set of leaf ndoes"""

    P = 2**d
    L_c = set(range(1, 2**(d-1)))
    L = set(range(2**(d-1), P))

    # - ancestors: dictionary {leaf nodes: {set of ancestors}}
    ancestors = {}
    for p in L:
        ancestors[p] = [math.floor(p/(2**j)) for j in range(1, d)]

        # Alternative way of retrieving right/left ancestors
    tree = Tree(d)
    right_left = tree.get_right_left()


    """
    - C[p]: finite set of cuts on node p--determined apriori {(l, theta)}
        Representation: dictionary {non-leaf node: list of (l, theta)}
        Require a list instead of set so it's ordered (indexed easily)


    From Kallus' paper, he wants us to:
    1. For each l in [d], sort data along x_l
    2a. For each non-leaf node, pick #features from [d] randomly
    2b. Set J = {1, (n-1/#cuts), ..., n-1} in decreasing order of cuts
    2c. Set Cp = {(l, midpoint between the two buckets in J) for all dimensions chosen and for all j in J}

    Make version of ALG3 to take all features"""


    if different_Cp:
        pass
    else:
        # -- BINARY COVARIATES --> Create a finite set of cuts for C for all features --
        C = []
        for i in range(len(train_X.columns)):
            C.append((i, 0))


    """ --- OTHER DATA --- 
    BIG M Constraints:
    - Ybar
    - Ymax
    - M

    BINARY ENCODING FOR CUTS
    - k_p: dictionary {non-leaf node p: k_p value}
    - Z_p: dictionary {non-leaf node p: k_p x |C_p| 2d matrix}
    """

    # ---- Big M Constraints ----
    # Ybar is merged into data, but it could not be. ybar is a numpy array
    minimum = min(train_y)

    ybar = train_y - minimum
    #data['ybar'] = ybar

    # Ymax
    ymax = max(ybar)

    # M
    # Find all sums for treatments 1, ..., m
    #treatment_counts = treatment.value_counts().to_list()
    unique, counts = np.unique(train_t, return_counts=True)
    #frequencies = numpy.asarray((unique, counts)).T
    M = np.array(counts)
    M -= len(L) * n_min
    M = max(M)
    
    model = gp.Model("Kallus")
    model.params.TimeLimit = 3600

    # -- VARIABLE DECLARATION --

    # -- Variables to determine: gamma and lambda --
    # 1. gamma_p = choice of cut at node p ([0, 1]^C_p) (only applies to non-leaf node)
    #       - represent with a matrix gamma (|L_c| x |C_p|)

    gamma = model.addVars(L_c, len(C), vtype=GRB.BINARY, name='gamma') 
    # This assumes gamma is binary
    #gamma = model.addVars(L_c, len(C), vtype=GRB.BINARY, name='gamma') 

    # 2. lambda_pt = choice of treatment t at node p (only applies to leaf nodes L)
    #       - represent with a matrix lamb (|L| x m)
    lamb = model.addVars(L, m, vtype=GRB.BINARY, name='lamb')


    # -- Other Variables in Formulation --
    # 1. w_ip = membership of datapoint i in node p (only applies to leaf nodes L)
    #       - represent with a matrix w (n x |L|)

    w = model.addVars(n, L, lb=0, ub=1, name='w')
    # This assumes w is binary, when in reality it is continuous from 0-1
    #w = model.addVars(n, L, vtype=GRB.BINARY, name='w') # Original paper has this be a continuous variable

    # 2. mu_p = mean outcome of prescribed treatment in node p
    #       - represent with a matrix mu (|L|)
    mu = model.addVars(L, lb=0, name='mu') # define in constraint

    # 3. nu_ip = "effect" of treatment in node p by multiplying mu and w
    #       - represent with a matrix nu (n x |L|)
    nu = model.addVars(n, L, lb=0, name='nu')

    # 4. delta_p = forces only 1 choice of cut at node p
    #       - represent with a dictionary {non-leaf node p: 1d matrix of size k_p}
    #delta = model.addVars(L, k, vtype=GRB.BINARY, name='delta')

    # 5. Chi_i(gamma) = 1 if choice of cut induces datapoint i to go left on the cut gamma, 0 otherwise
    chi = model.addVars(L_c, n, vtype=GRB.BINARY, name='chi')

    if bertsimas:
        # 6. f_i
        f = model.addVars(n, lb=0, name='f')

        # 7. Beta_lt
        beta = model.addVars(L, m, lb=0, name='beta')

    theta = 0.5
    
        # --- OBJECTIVE FUNCTION ---
    if bertsimas:
        model.setObjective(theta * gp.quicksum(nu[i, p] for i in range(n) for p in L) 
                       - (1-theta) * gp.quicksum((train_y[i] - f[i]) * (train_y[i] - f[i]) for i in range(n)), GRB.MAXIMIZE)
    else:
        model.setObjective(gp.quicksum(nu[i, p] for i in range(n) for p in L), GRB.MAXIMIZE)


    # --- CONSTRAINTS ---
    # Constraint 4c (4b is done by definition of variables)
    if delta_include:
        for p in L_c:
            # need to do matrix multiplication somehow, but this might work?
            for j in range(k):
                model.addConstr(delta[p, j] == gp.quicksum(gamma[p, i] * z[j, i] for i in range(len(C))))

    # Additional constraint that gamma[p] adds up to 1
    # CHECKED
    for p in L_c:
        model.addConstr(gp.quicksum(gamma[p, i] for i in range(len(C))) == 1)

    # Add constraint Chi
    for i in range(n):
        for p in L_c:
            model.addConstr(chi[p, i] == gp.quicksum(gamma[p, j] for j in range(len(C)) if C[j][1] >= train_X.iloc[i, C[j][0]]))


    # Constraint 4d&e (Membership restriction from its ancestors) CHECKED
    for p in L:
        A_p = ancestors[p] #index ancestors of p
        for q in A_p:
            R_pq = right_left[(p, q)]
            for i in range(n):
                model.addConstr(w[i, p] <= (1+R_pq)/2 - R_pq * chi[q, i])


    #4e CHECKED
    for p in L:
        A_p = ancestors[p] #index ancestors of p
        for i in range(n):
            model.addConstr(w[i, p] >= 1 + gp.quicksum(-chi[q, i] for q in A_p if right_left[(p, q)] == 1)
                        + gp.quicksum(-1+chi[q, i] for q in A_p if right_left[(p, q)] == -1))


    # Constraint 4f
    # CHECKED
    for t in m:
        for p in L:
            model.addConstr(gp.quicksum(w[i, p] for i in range(n) if train_t[i] == t) >= n_min) #assuming the input comes in vector (Xi, Ti, Yi)
            # only add the datapoints that have been given treatment t

    # Constraints 4g&h (Linearization of nu)
    # CHECKED
    for p in L:
        for i in range(n):
            model.addConstr(nu[i, p] <= ymax * w[i, p])
            model.addConstr(nu[i, p] <= mu[p])
            model.addConstr(nu[i, p] >= mu[p] - ymax * (1-w[i, p]))

    # Constraint 4i (Choice of treatment applied to p)
    # CHECKED
    for p in L:
        model.addConstr(gp.quicksum(lamb[p, t] for t in m) == 1)

    # Constraint 4j&k (Consistency between lambda and mu)
    # CHECKED. There are some inconsistencies where some w don't appear, but this is because ybar is 0 (i.e. the minimum)
    for p in L:
        for t in m:
            model.addConstr(gp.quicksum(nu[i, p] - w[i, p] * ybar[i] for i in range(n) if train_t[i] == t) <= M*(1-lamb[p, t]))
            model.addConstr(gp.quicksum(nu[i, p] - w[i, p] * ybar[i] for i in range(n) if train_t[i] == t) >= M*(lamb[p, t]-1))
    #model.addConstr(lamb[2, 0] == 1)
    if bertsimas:
        for i in range(n):
            for p in L:
                for t in m:
                    if train_t[i] == t:
                        model.addConstr(f[i] - beta[p, t] <= M * (1-w[i, p]))
                        model.addConstr(f[i] - beta[p, t] >= M * (w[i, p]-1))

    model.optimize()
    g = model.getAttr("X", gamma).items()
    l = model.getAttr("X", lamb).items()
    
    branching = {i[0][0]: i[0][1] for i in g if i[1] == 1.0}
    treatments = {i[0][0]: i[0][1] for i in l if i[1] == 1.0}
    
    return branching, treatments


In [7]:
"""kallus = pd.DataFrame(columns=['Dataset','Depth','P(Correct Treatment)','Test Error','Test % Optimal',
                               'Test % Same Treatment', 'Train Error', 'Train % Optimal', 
                               'Train % Same Treatment', 'Tree'])
bertsimas = pd.DataFrame(columns=['Dataset','Depth','P(Correct Treatment)','Test Error','Test % Optimal',
                               'Test % Same Treatment', 'Train Error', 'Train % Optimal', 
                                  'Train % Same Treatment', 'Tree'])"""


#bertsimas = pd.read_csv('bertsimas_athey_500.csv')

FileNotFoundError: [Errno 2] File b'bertsimas_athey_500.csv' does not exist: b'bertsimas_athey_500.csv'

In [10]:
kallus = pd.DataFrame(columns=['Dataset','Depth','P(Correct Treatment)','Test Error','Test % Optimal',
                               'Test % Same Treatment', 'Train Error', 'Train % Optimal', 
                               'Train % Same Treatment', 'Tree'])

d = 2
prob = 0.5
dataset = 3
bert = False

depths = [1,2,3]
datasets = [1, 2, 3, 4, 5]
probs = [0.1, 0.5, 0.9]

for dataset in datasets:
    for d in depths:
        for prob in probs:
            train_filepath = 'data/Athey_v1_N_500/data_train_enc_' + str(prob) + '_' + str(dataset) + '.csv'
            test_filepath = 'data/Athey_v1_N_500/data_test_enc_' + str(prob) + '_' + str(dataset) + '.csv'
            train_X, train_t, train_y, train_real = open_file(train_filepath)
            branching, treatments = run_tree(d, bert)

            tree = Tree(d)
            L = set(range(2**(d-1), 2**d))
            test_X, test_t, test_y, test_real = open_file(test_filepath)
            error, pct, acc = get_metrics(test_X, test_real, test_t)
            error1, pct1, acc1 = get_metrics(train_X, train_real, train_t)

            tree_stats = 'branching = ' + str(branching) + ', treatments = ' + str(treatments)
            row = [dataset, d, prob, error, pct*100, acc*100, error1, pct1*100, acc1*100, tree_stats]
            print(row)
            if bert:
                bertsimas.loc[len(bertsimas)] = row
            else:
                kallus.loc[len(kallus)] = row


Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2007 rows, 1003 columns and 6504 nonzeros
Model fingerprint: 0xe4d30c4c
Variable types: 1001 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [2e-02, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Presolve removed 2007 rows and 1003 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 269.252 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.692522919642e+02, best bound 2.692522919642e+02, gap 0.0000%
[1, 1, 0.1, 179.61433872965205, 56.57, 49.71, 25.77127660053485, 28.999999999999996, 51.800000000000004, 'br


Solution count 7: 282.762 280.474 277.617 ... 273.457

Optimal solution found (tolerance 1.00e-04)
Best objective 2.827622527092e+02, best bound 2.827622527092e+02, gap 0.0000%
[1, 2, 0.5, 30.545715035334094, 83.05, 50.36000000000001, 4.905546780062993, 73.6, 48.8, 'branching = {1: 4}, treatments = {2: 1, 3: 0}']
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5515 rows, 2526 columns and 21099 nonzeros
Model fingerprint: 0xf36ab076
Variable types: 2002 continuous, 524 integer (524 binary)
Coefficient statistics:
  Matrix range     [2e-02, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Presolve removed 4624 rows and 2146 columns
Presolve time: 0.07s
Presolved: 891 rows, 380 columns, 3880 nonzeros
Variable types: 264 continuous, 1

Model fingerprint: 0xa05c66bb
Variable types: 4004 continuous, 1568 integer (1568 binary)
Coefficient statistics:
  Matrix range     [2e-02, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Presolve removed 1368 rows and 1270 columns
Presolve time: 0.15s
Presolved: 12163 rows, 4302 columns, 38564 nonzeros
Variable types: 3956 continuous, 346 integer (346 binary)
Found heuristic solution: objective 272.9423009
Found heuristic solution: objective 273.4574685

Root relaxation: objective 9.462341e+02, 7373 iterations, 0.52 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  946.23407    0  274  273.45747  946.23407   246%     -    0s
     0     0  846.14062    0  298  273.45747  846.14062   209%     -    1s
     0     0  846.14062    0  274  273.45747  846.14062   209%     -    2s
     0     0  590.50000    0  

     0     0  524.20616    0  265  303.08288  524.20616  73.0%     -    3s
     0     0  514.20188    0  300  303.08288  514.20188  69.7%     -    3s
     0     0  513.83681    0  298  303.08288  513.83681  69.5%     -    3s
     0     0  513.83681    0  298  303.08288  513.83681  69.5%     -    3s
H    0     0                     304.4188581  513.83681  68.8%     -    4s
     0     0  512.25000    0  298  304.41886  512.25000  68.3%     -    4s
     0     0  512.25000    0  298  304.41886  512.25000  68.3%     -    4s
     0     0  512.25000    0  298  304.41886  512.25000  68.3%     -    4s
     0     0  512.25000    0   78  304.41886  512.25000  68.3%     -    5s
     0     2  512.25000    0   78  304.41886  512.25000  68.3%     -    5s
H   62    66                     305.1206406  500.00000  63.9%   190    6s
H  140   116                     305.9498815  500.00000  63.4%   182    7s
*  256   203              18     306.1015805  500.00000  63.3%   185    8s
*  258   203             


Cutting planes:
  Gomory: 2
  Implied bound: 13
  MIR: 6
  Flow cover: 3
  RLT: 88
  Relax-and-lift: 6
  BQP: 5

Explored 1 nodes (1466 simplex iterations) in 0.66 seconds
Thread count was 8 (of 8 available processors)

Solution count 4: 245.231 243.557 236.23 203.116 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.452306291343e+02, best bound 2.452306291343e+02, gap 0.0000%
[2, 2, 0.1, 63.74597884426453, 73.06, 25.95, 4.147362670960025, 70.19999999999999, 25.2, 'branching = {1: 6}, treatments = {2: 1, 3: 0}']
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5515 rows, 2526 columns and 21121 nonzeros
Model fingerprint: 0x67aedb6e
Variable types: 2002 continuous, 524 integer (524 binary)
Coefficient statistics:
  Matrix range     [2e-03, 3e+02]
  Objective range  [1e+00, 1e+00

...

 59775  1156  306.08435   25   78  305.06751  324.40419  6.34%  68.4  205s

Cutting planes:
  Gomory: 49
  Implied bound: 1
  MIR: 530
  Flow cover: 12
  Inf proof: 1
  RLT: 915
  BQP: 45

Explored 63470 nodes (4232239 simplex iterations) in 209.27 seconds
Thread count was 8 (of 8 available processors)

Solution count 10: 305.068 305.012 304.708 ... 270.979

Optimal solution found (tolerance 1.00e-04)
Best objective 3.050675100280e+02, best bound 3.050675100280e+02, gap 0.0000%
[4, 3, 0.5, 54.53271149738661, 74.98, 49.3, 1.620030453770224, 84.0, 46.2, 'branching = {1: 10, 2: 6, 3: 0}, treatments = {4: 1, 5: 0, 6: 1, 7: 0}']
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 13531 rows, 5572 columns and 53163 nonzeros
Model fingerprint: 0x564ee7c9
Variable types: 4004 continuous, 1568 integer (


Solution count 1: 283.554 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.835537186181e+02, best bound 2.835537186181e+02, gap 0.0000%
[5, 1, 0.5, 348.7848341374893, 34.33, 49.24, 17.3062281572362, 44.2, 54.6, 'branching = {}, treatments = {1: 1}']
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2007 rows, 1003 columns and 6504 nonzeros
Model fingerprint: 0xbd78f0f8
Variable types: 1001 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e-02, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Presolve removed 2007 rows and 1003 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available proces

H    0     0                     310.2796788  356.42091  14.9%     -    0s
H    0     0                     311.1543841  356.42091  14.5%     -    0s
     0     0  312.22014    0  104  311.15438  312.22014  0.34%     -    0s
     0     0  312.22014    0    2  311.15438  312.22014  0.34%     -    0s
     0     0  312.22014    0   80  311.15438  312.22014  0.34%     -    0s
     0     0  312.22014    0  101  311.15438  312.22014  0.34%     -    0s
     0     0  312.22014    0   94  311.15438  312.22014  0.34%     -    0s
     0     0  312.22014    0   94  311.15438  312.22014  0.34%     -    0s
     0     0  312.22014    0  103  311.15438  312.22014  0.34%     -    0s
     0     0  312.22014    0  103  311.15438  312.22014  0.34%     -    0s
     0     2  312.22014    0  103  311.15438  312.22014  0.34%     -    0s

Cutting planes:
  Gomory: 1
  MIR: 2
  Flow cover: 2
  RLT: 8

Explored 108 nodes (6318 simplex iterations) in 0.80 seconds
Thread count was 8 (of 8 available processors)

So

H  157   114                     291.2988135  505.00000  73.4%   147    6s
H  296   155                     291.3015181  502.00000  72.3%   131    7s
H  329   171                     291.3015201  502.00000  72.3%   127    7s
   637   345  321.89436   12   86  291.30152  500.00000  71.6%   144   10s
H 1071   522                     291.3015381  473.74203  62.6%   150   13s
  1287   619  320.84479   18   32  291.30154  461.41798  58.4%   152   15s
* 1658   774              11     292.8837352  442.96801  51.2%   145   16s
H 1749   800                     292.8837475  438.56837  49.7%   143   17s
H 1770   797                     293.3337589  438.21166  49.4%   142   17s
  2095   907  327.96340   16  274  293.33376  431.93155  47.2%   139   21s
  2110   917  412.40171   12  279  293.33376  431.93155  47.2%   138   25s
  2113   922  431.93155   14  286  293.33376  431.93155  47.2%   143   31s
  2223   995  361.19122   22  144  293.33376  431.93155  47.2%   152   35s
  2537  1087  299.70729  

* 1588   681              39     311.6907181  487.87732  56.5%   146   33s
H 1616   651                     314.4022317  487.87732  55.2%   146   34s
  1625   664  396.35694   32   75  314.40223  487.87732  55.2%   146   35s
H 2145   682                     315.3805584  473.73908  50.2%   139   38s
H 2151   657                     315.5141369  473.73908  50.1%   139   38s
  2382   782  370.33809   26   50  315.51414  469.97614  49.0%   134   40s
* 3345  1025              45     317.2681814  456.47024  43.9%   119   43s
  3596  1118 infeasible   33       317.26818  454.93197  43.4%   118   47s
  3743  1238  442.51456   24  227  317.26818  454.16851  43.1%   117   50s
  5182  1916  423.41056   27  130  317.26818  444.13522  40.0%   108   55s
  6241  2322  400.96234   32  147  317.26818  439.69687  38.6%   107   60s
  7177  2623  428.54605   30  112  317.26818  435.56081  37.3%   104   66s
  8433  3089  330.36357   41   31  317.26818  430.32272  35.6%   100   70s
  9667  3392  399.28685  

In [13]:
kallus.to_csv('summary_results/kallus_v1_500.csv')

In [11]:
prob = 0.9
dataset = 4
d = 3
bert = False
train_filepath = 'data/Athey_v1_N_500/data_train_enc_' + str(prob) + '_' + str(dataset) + '.csv'
test_filepath = 'data/Athey_v1_N_500/data_test_enc_' + str(prob) + '_' + str(dataset) + '.csv'
train_X, train_t, train_y, train_real = open_file(train_filepath)
branching, treatments = run_tree(d, bert)

tree = Tree(d)
L = set(range(2**(d-1), 2**d))
test_X, test_t, test_y, test_real = open_file(test_filepath)
error, pct, acc = get_metrics(test_X, test_real, test_t)
error1, pct1, acc1 = get_metrics(train_X, train_real, train_t)

tree_stats = 'branching = ' + str(branching) + ', treatments = ' + str(treatments)
row = [dataset, d, prob, error, pct*100, acc*100, error1, pct1*100, acc1*100, tree_stats]
print(row)

Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 13531 rows, 5572 columns and 53163 nonzeros
Model fingerprint: 0x564ee7c9
Variable types: 4004 continuous, 1568 integer (1568 binary)
Coefficient statistics:
  Matrix range     [2e-01, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Presolve removed 1320 rows and 1251 columns
Presolve time: 0.15s
Presolved: 12211 rows, 4321 columns, 38709 nonzeros
Variable types: 3972 continuous, 349 integer (349 binary)
Found heuristic solution: objective 301.3276415

Root relaxation: objective 9.436737e+02, 8759 iterations, 0.95 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0

In [76]:
tree = Tree(d)
L = set(range(2**(d-1), 2**d))
test_X, test_t, test_y, test_real = open_file(test_filepath)
error, pct, acc = get_metrics(test_X, test_real, test_t)
error1, pct1, acc1 = get_metrics(train_X, train_real, train_t)
print(error, pct, acc)
print(error1, pct1, acc1)

267.9863312540136 0.4446 0.5066
21.252969854711775 0.354 0.5


In [77]:
tree_stats = 'branching = ' + str(branching) + ', treatments = ' + str(treatments)
row = [dataset, d, prob, error, pct*100, acc*100, error1, pct1*100, acc1*100, tree_stats]
if bert:
    bertsimas.loc[len(bertsimas)] = row
else:
    kallus.loc[len(kallus)] = row

[5, 3, 0.9, 267.9863312540136, 44.46, 50.660000000000004, 21.252969854711775, 35.4, 50.0, 'branching = {1: 10, 2: 16, 3: 0}, treatments = {4: 1, 5: 1, 6: 1, 7: 1}']


In [78]:
print(bertsimas)
bertsimas.to_csv('bertsimas_athey_500.csv', index=False)

    Dataset  Depth  P(Correct Treatment)  Test Error  Test % Optimal  \
0         1      2                   0.5   53.227602           76.83   
1         1      3                   0.5  123.833930           66.00   
2         1      3                   0.5  123.833930           66.00   
3         2      2                   0.5   24.915258           84.31   
4         2      3                   0.5   24.915258           84.31   
5         3      2                   0.5  166.280038           58.61   
6         3      3                   0.5   57.544135           77.50   
7         4      2                   0.5  432.810829           24.05   
8         4      3                   0.5  231.607156           44.69   
9         5      2                   0.5  189.645268           55.55   
10        5      3                   0.5   36.072978           82.59   
11        1      2                   0.9  224.473595           49.78   
12        1      3                   0.9  224.473595           4

### Summary of Performance for Athey's Synthetic Data

In [247]:
# SUMMARY: KALLUS on Athey
summary = {'Method': ['Dataset 1', 'Dataset 1', 'Dataset 2', 'Dataset 2', 'Dataset 3',
                     'Dataset 3', 'Dataset 4', 'Dataset 4', 'Dataset 5', 'Dataset 5', 'Dataset 1', 
                      'Dataset 1', 'Dataset 2', 'Dataset 2', 'Dataset 3',
                     'Dataset 3', 'Dataset 4', 'Dataset 4', 'Dataset 5', 'Dataset 5'],
           'Depth': [2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3],
           'P(Correct Treatment)': ['0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.9', '0.9',
                                   '0.9', '0.9', '0.9', '0.9', '0.9', '0.9', '0.9', '0.9'],
          'Error': [74.67, 74.67, 46.40, 79.86, 145, 109.21, 18.15, 18.15, 152.55, 90.47, 199.70, 238.67, 219.02, 
                    239.10, 201.21, 201.21, 99.23, 88.12, 196.07, 156.84],
          '% Classified': [71.97, 71.97, 78.49, 71.11, 61.22, 68.03, 88.33, 88.33, 60.41, 70.03, 53.35,
                          47.49, 49.91, 47.49, 53.42, 53.42, 71.19, 74.11, 54.12, 58.02]}
summary = pd.DataFrame(summary)
summary.to_csv('kallus_tree_athey.csv', index=False)

In [248]:
# SUMMARY: Bertsimas on Athey
summary = {'Method': ['Dataset 1', 'Dataset 1', 'Dataset 2', 'Dataset 2', 'Dataset 3',
                     'Dataset 3', 'Dataset 4', 'Dataset 4', 'Dataset 5', 'Dataset 5', 'Dataset 1', 
                      'Dataset 1', 'Dataset 2', 'Dataset 2', 'Dataset 3',
                     'Dataset 3', 'Dataset 4', 'Dataset 4', 'Dataset 5', 'Dataset 5'],
           'Depth': [2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3],
           'P(Correct Treatment)': ['0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.9', '0.9',
                                   '0.9', '0.9', '0.9', '0.9', '0.9', '0.9', '0.9', '0.9'],
          'Error': [74.67, 74.67, 46.40, 55.17, 145, 109.21, 18.15, 18.15, 152.55, 90.47, 199.70, 154.07, 219.02, 
                    180.83, 201.21, 116.75, 99.23, 88.12, 196.07, 156.84],
          '% Classified': [71.97, 71.97, 78.49, 76.49, 61.22, 68.03, 88.33, 88.33, 60.41, 70.03, 53.35,
                          62.17, 49.91, 53.91, 53.42, 71.59, 71.19, 74.11, 54.12, 58.02]}
summary = pd.DataFrame(summary)
summary.to_csv('bertsimas_tree_athey.csv', index=False)


In [37]:
def datapoint_tree_avg(node, i):
    if node in L: #if datapoint has reached leaf node, calculate error
        return node
    if train_X.iloc[i, branching[node]] <= 0: # go left (node 2)
        return datapoint_tree_avg(tree.get_left_children(node), i)
    else:
        return datapoint_tree_avg(tree.get_right_children(node), i)

In [ ]:
summation = {20: 0, 21: 0, 30: 0, 31: 0}
count = {20: 0, 21: 0, 30: 0, 31: 0}

for i in range(n):
    leaf_node = datapoint_tree_avg(1, i)
    index = str(leaf_node) + str(train_t[i])
    summation[int(index)] += train_y[i]
    count[int(index)] += 1
    
avg = {}
for i in summation:
    avg[i] = float(summation[i]) / count[i]

print(avg)

In [49]:
summation = {2: 0, 3: 0}
count = {2: 0, 3: 0}

for i in range(n):
    leaf_node = datapoint_tree_avg(1, i)
    if train_t[i] == treatments[leaf_node]:
        summation[leaf_node] += train_y[i]
        count[leaf_node] += 1
    
avg = {}
for i in summation:
    avg[i] = float(summation[i]) / count[i]

print(avg)

{2: 0.39851651102745433, 3: 0.34277517675515534}


In [59]:
N = L.union(L_c)
print(N)

model = gp.Model("Nathan")

# -- VARIABLE DECLARATION --

# -- Variables to determine: gamma and lambda --
# 1. gamma_p = choice of cut at node p ([0, 1]^C_p) (only applies to non-leaf node)
#       - represent with a matrix gamma (|L_c| x |C_p|)

gamma = model.addVars(L_c, len(C), vtype=GRB.BINARY, name='gamma') 
# This assumes gamma is binary
#gamma = model.addVars(L_c, len(C), vtype=GRB.BINARY, name='gamma') 

# 2. lambda_pt = choice of treatment t at node p (only applies to leaf nodes L)
#       - represent with a matrix lamb (|L| x m)
lamb = model.addVars(L, m, vtype=GRB.BINARY, name='lamb')


# -- Other Variables in Formulation --
# 1. w_ip = membership of datapoint i in node p (only applies to leaf nodes L)
#       - represent with a matrix w (n x |L|)

c = model.addVars(n, N, vtype=GRB.BINARY, name='c')
# This assumes w is binary, when in reality it is continuous from 0-1
#w = model.addVars(n, L, vtype=GRB.BINARY, name='w') # Original paper has this be a continuous variable

# 2. mu_p = mean outcome of prescribed treatment in node p
#       - represent with a matrix mu (|L|)
v = model.addVars(n, L_c, vtype=GRB.BINARY, name='v') # define in constraint

# 3. nu_ip = "effect" of treatment in node p by multiplying mu and w
#       - represent with a matrix nu (n x |L|)
w = model.addVars(n, vtype=GRB.BINARY, name='w')

{1, 2, 3}


In [ ]:


"""if different_Cp:
# ---- K and Z if we followed the definition of C_p from the paper -----
    for p in L_c:
        k[p] = math.ceil(math.log2(len(C[p])))

    z = {}
    for p in L_c:
        matrix = np.zeros((k[p], len(C[p])))
        print(matrix.shape)
        for i in range(1, k[p]+1):
            for j in range(1, len(C[p])+1):
                if math.floor(j/(2**i)) % 2 == 1: # odd number
                    z[i-1, j-1] = 1
                else:
                    z[i-1, j-1] = 0
        z[p] = matrix

else:
    # ---- K and Z if we had constant C for all nodes ----
    k = math.ceil(math.log2(len(C)))
    z = np.zeros((k, len(C)))
    for i in range(1, k+1):
        for j in range(1, len(C)+1):
            if math.floor(j/(2**i)) % 2 == 1: # odd number
                z[i-1, j-1] = 1
            else:
                z[i-1, j-1] = 0
print(z)"""